In [2]:
# Downloading all dependencies and starting spark session
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
import findspark
findspark.init()
findspark.find()

import pyspark
from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,891 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,773 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,207 kB]
Hit:13 https://ppa.launchpadcontent.ne

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip '/content/drive/MyDrive/CDAC Project/archive.zip' -d '/content/unzipped_data'

Archive:  /content/drive/MyDrive/CDAC Project/archive.zip
  inflating: /content/unzipped_data/factory_sensor_simulator_2040.csv  


In [28]:
df = spark.read.csv('/content/unzipped_data/factory_sensor_simulator_2040.csv', header=True, inferSchema=True)

In [29]:
df.show()

+----------+--------------------+-----------------+-----------------+-------------+-------------+--------+-------------+-----------------+--------------------+-------------------------+-------------------------+---------------------+--------------+------------------------+--------------------------+---------------------+---------------+----------------------+------------------+----------+------------------+
|Machine_ID|        Machine_Type|Installation_Year|Operational_Hours|Temperature_C|Vibration_mms|Sound_dB|Oil_Level_pct|Coolant_Level_pct|Power_Consumption_kW|Last_Maintenance_Days_Ago|Maintenance_History_Count|Failure_History_Count|AI_Supervision|Error_Codes_Last_30_Days|Remaining_Useful_Life_days|Failure_Within_7_Days|Laser_Intensity|Hydraulic_Pressure_bar|Coolant_Flow_L_min|Heat_Index|AI_Override_Events|
+----------+--------------------+-----------------+-----------------+-------------+-------------+--------+-------------+-----------------+--------------------+-------------------

In [30]:
df.printSchema()

root
 |-- Machine_ID: string (nullable = true)
 |-- Machine_Type: string (nullable = true)
 |-- Installation_Year: integer (nullable = true)
 |-- Operational_Hours: integer (nullable = true)
 |-- Temperature_C: double (nullable = true)
 |-- Vibration_mms: double (nullable = true)
 |-- Sound_dB: double (nullable = true)
 |-- Oil_Level_pct: double (nullable = true)
 |-- Coolant_Level_pct: double (nullable = true)
 |-- Power_Consumption_kW: double (nullable = true)
 |-- Last_Maintenance_Days_Ago: integer (nullable = true)
 |-- Maintenance_History_Count: integer (nullable = true)
 |-- Failure_History_Count: integer (nullable = true)
 |-- AI_Supervision: boolean (nullable = true)
 |-- Error_Codes_Last_30_Days: integer (nullable = true)
 |-- Remaining_Useful_Life_days: double (nullable = true)
 |-- Failure_Within_7_Days: boolean (nullable = true)
 |-- Laser_Intensity: double (nullable = true)
 |-- Hydraulic_Pressure_bar: double (nullable = true)
 |-- Coolant_Flow_L_min: double (nullable = tr

In [31]:
df.columns

['Machine_ID',
 'Machine_Type',
 'Installation_Year',
 'Operational_Hours',
 'Temperature_C',
 'Vibration_mms',
 'Sound_dB',
 'Oil_Level_pct',
 'Coolant_Level_pct',
 'Power_Consumption_kW',
 'Last_Maintenance_Days_Ago',
 'Maintenance_History_Count',
 'Failure_History_Count',
 'AI_Supervision',
 'Error_Codes_Last_30_Days',
 'Remaining_Useful_Life_days',
 'Failure_Within_7_Days',
 'Laser_Intensity',
 'Hydraulic_Pressure_bar',
 'Coolant_Flow_L_min',
 'Heat_Index',
 'AI_Override_Events']

In [32]:
row_count = df.count()
cloumn_count = len(df.columns)
print('shape :' ,(row_count, cloumn_count))

shape : (500000, 22)


In [33]:
df = df.drop('Machine_ID')

In [34]:
df = df.dropDuplicates()

In [35]:
row_count = df.count()
cloumn_count = len(df.columns)
print('shape :' ,(row_count, cloumn_count))

shape : (500000, 21)


In [36]:
from pyspark.sql.types import IntegerType, DoubleType, FloatType, StringType
from pyspark.sql.functions import col, mean, sum, isnan, when
from pyspark.ml.feature import Imputer

In [37]:
# Identify numeric and categorical columns
numeric_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, (IntegerType, DoubleType, FloatType))]
categorical_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, StringType)]

In [38]:
print(numeric_cols)
print(categorical_cols)

['Installation_Year', 'Operational_Hours', 'Temperature_C', 'Vibration_mms', 'Sound_dB', 'Oil_Level_pct', 'Coolant_Level_pct', 'Power_Consumption_kW', 'Last_Maintenance_Days_Ago', 'Maintenance_History_Count', 'Failure_History_Count', 'Error_Codes_Last_30_Days', 'Remaining_Useful_Life_days', 'Laser_Intensity', 'Hydraulic_Pressure_bar', 'Coolant_Flow_L_min', 'Heat_Index', 'AI_Override_Events']
['Machine_Type']


In [39]:
# Create a list of expressions to count nulls and NaNs in each column
null_nan_counts = []
for c in df.columns:
    if c in numeric_cols:
        null_nan_counts.append(
            sum(when(col(c).isNull() | isnan(c), 1).otherwise(0)).alias(c)
        )
    else:
        null_nan_counts.append(
            sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
        )

# Execute the aggregation to get the counts
null_nan_counts_df = df.agg(*null_nan_counts)

# Display the results
null_nan_counts_df.show()

+------------+-----------------+-----------------+-------------+-------------+--------+-------------+-----------------+--------------------+-------------------------+-------------------------+---------------------+--------------+------------------------+--------------------------+---------------------+---------------+----------------------+------------------+----------+------------------+
|Machine_Type|Installation_Year|Operational_Hours|Temperature_C|Vibration_mms|Sound_dB|Oil_Level_pct|Coolant_Level_pct|Power_Consumption_kW|Last_Maintenance_Days_Ago|Maintenance_History_Count|Failure_History_Count|AI_Supervision|Error_Codes_Last_30_Days|Remaining_Useful_Life_days|Failure_Within_7_Days|Laser_Intensity|Hydraulic_Pressure_bar|Coolant_Flow_L_min|Heat_Index|AI_Override_Events|
+------------+-----------------+-----------------+-------------+-------------+--------+-------------+-----------------+--------------------+-------------------------+-------------------------+---------------------+--

In [40]:
# Impute numeric columns with the mean
imputer = Imputer(inputCols=numeric_cols, outputCols=numeric_cols, strategy="mean")
df = imputer.fit(df).transform(df)

In [41]:
null_nan_counts = []
for c in df.columns:
    if c in numeric_cols:
        null_nan_counts.append(
            sum(when(col(c).isNull() | isnan(c), 1).otherwise(0)).alias(c)
        )
    else:
        null_nan_counts.append(
            sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
        )

null_nan_counts_df = df.agg(*null_nan_counts)

null_nan_counts_df.show()

+------------+-----------------+-----------------+-------------+-------------+--------+-------------+-----------------+--------------------+-------------------------+-------------------------+---------------------+--------------+------------------------+--------------------------+---------------------+---------------+----------------------+------------------+----------+------------------+
|Machine_Type|Installation_Year|Operational_Hours|Temperature_C|Vibration_mms|Sound_dB|Oil_Level_pct|Coolant_Level_pct|Power_Consumption_kW|Last_Maintenance_Days_Ago|Maintenance_History_Count|Failure_History_Count|AI_Supervision|Error_Codes_Last_30_Days|Remaining_Useful_Life_days|Failure_Within_7_Days|Laser_Intensity|Hydraulic_Pressure_bar|Coolant_Flow_L_min|Heat_Index|AI_Override_Events|
+------------+-----------------+-----------------+-------------+-------------+--------+-------------+-----------------+--------------------+-------------------------+-------------------------+---------------------+--

In [42]:
from pyspark.sql import functions as F

In [43]:
max_installation_year = df.agg(F.max("Installation_Year")).collect()[0][0]
print(f"Maximum Installation Year: {max_installation_year}")

Maximum Installation Year: 2040


In [44]:
# 1. Calculate Machine Age from Installation_Year
df = df.withColumn("Machine_Age", F.lit(2040) - F.col("Installation_Year"))

In [45]:
# 2. Combine oil and coolant level to infer maintenance status (simple average)
df = df.withColumn("Avg_Fluid_Level", (F.col("Oil_Level_pct") + F.col("Coolant_Level_pct")) / 2)

In [46]:
# 3. Flag if AI supervision is active (boolean to int)
df = df.withColumn("AI_Supervision_Flag", F.col("AI_Supervision").cast("int"))

In [47]:
# 4. Encode categorical variable 'Machine_Type' (StringIndexer)
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Machine_Type", outputCol="Machine_Type_Index")
df = indexer.fit(df).transform(df)

In [48]:
df.show()

+--------------------+-----------------+-----------------+-------------+-------------+--------+-------------+-----------------+--------------------+-------------------------+-------------------------+---------------------+--------------+------------------------+--------------------------+---------------------+-----------------+----------------------+------------------+-----------------+------------------+-----------+------------------+-------------------+------------------+
|        Machine_Type|Installation_Year|Operational_Hours|Temperature_C|Vibration_mms|Sound_dB|Oil_Level_pct|Coolant_Level_pct|Power_Consumption_kW|Last_Maintenance_Days_Ago|Maintenance_History_Count|Failure_History_Count|AI_Supervision|Error_Codes_Last_30_Days|Remaining_Useful_Life_days|Failure_Within_7_Days|  Laser_Intensity|Hydraulic_Pressure_bar|Coolant_Flow_L_min|       Heat_Index|AI_Override_Events|Machine_Age|   Avg_Fluid_Level|AI_Supervision_Flag|Machine_Type_Index|
+--------------------+-----------------+--

In [49]:
# To drop multiple columns
columns_to_drop = ["Machine_Type", "Oil_Level_pct", "Coolant_Level_pct", "AI_Supervision"]
df = df.drop(*columns_to_drop)

In [50]:
cloumn_count = len(df.columns)
cloumn_count

21

In [51]:
# Defining the path to save the CSV file
output_path = "/content/preprocessed_data.csv"

# Using 'header=True' to include the header row in the CSV
# Using 'mode="overwrite"' to overwrite the file if it already exists
df.write.csv(output_path, header=True, mode="overwrite")

print(f"DataFrame saved to {output_path}")

DataFrame saved to /content/preprocessed_data.csv
